In [ ]:
def extract_data():
    all_findings = []
    all_impressions = []
    all_img_names = []
    rids = []
    
    total_count = 0 # Count of reports available in the dataset
    no_image_count = 0 # Count of reports having no associated chest image
    no_impression_count = 0 # Count of reports having an empty "Impression" section
    no_findings_count = 0 # Count of reports having an empty "Findings" section

    # Storing impressions, findings and the image names in vectors
    for file in tqdm(os.listdir(annotation_folder)):
    # for file in tqdm(annotation_folder):
        total_count += 1
        file = os.path.abspath(annotation_folder) + '/' + file
        e = xml.etree.ElementTree.parse(file).getroot()

        rid = e.find('pmcId').get('id') # Report Id
        # We choose to ignore reports having no associated image
        image_id = e.find('parentImage')
        if image_id is None:
            no_image_count += 1
            continue

        image_id = image_id.get('id')
#         image_name = os.path.abspath('.') + '/' + image_id + '.png'
        image_name = image_folder + '/' + image_id + '.png'
        findings = ''
        impression = ''

        # Parsing "Impression" and "Findings"
        for element in e.findall('MedlineCitation/Article/Abstract/AbstractText'):
            if element.get('Label') == 'FINDINGS':
                findings = element.text
            if element.get('Label') == 'IMPRESSION':
                impression = element.text

        # Sanity check: Skip this report if it has an empty "Impression" section
        if findings is None:
            no_findings_count += 1
            #findings = 'No finding'
            continue
        if impression is None:
            no_impression_count += 1
            continue
        
        # Transforming findings and impressions into lists of sentences
        # https://stackoverflow.com/questions/21840389/python-regular-expression-remove-period-from-number-at-end-of-sentence
        findings = findings.replace("XXXX", "") #"XXXX" represents information anonymized
        findings=re.sub('((\d+)[\.])(?!([\d]+))','\g<2>',findings)
        findings = re.sub(" \d+", " ", findings)
#         sentences = findings.split('.')
        sentences = findings
#         del sentences[-1]
#         sentences = ['<start> ' + sentence + ' <end>' for sentence in sentences]
        # sentences = ['<start> ' + sentences + ' <end>']
        findings = sentences

        impression = impression.replace("XXXX", "") #"XXXX" represents information anonymized
        impression=re.sub('((\d+)[\.])(?!([\d]+))','\g<2>',impression)
        impression = re.sub(" \d+", " ", impression)
#https://www.tutorialspoint.com/How-to-remove-specific-characters-from-a-string-in-Python
        impression=impression.replace("1", "")
#         sentences = impression.split('.')
#         del sentences[-1]
        sentences = impression
#         sentences = ['<start> ' + sentence + ' <end>' for sentence in sentences]
        sentences = ['<start> ' + sentences + ' <end>' ]
        impression = sentences

        #appending to vectors
        all_img_names.append(image_name)
        all_findings.append(findings)
        all_impressions.append(impression)
        rids.append(rid)

    print("Number of reports available:", total_count)
    print("Number of reports selected:", len(all_img_names))
    print("Number of reports not having images (skipped):", no_image_count)
    print("Number of reports with Impression section empty (skipped):", no_impression_count)
    print("Number of reports with Findings section empty:", no_findings_count)
    print("Total skipped:", no_image_count + no_impression_count + no_findings_count)

    return all_findings, all_impressions, all_img_names, rids

all_findings, all_impressions, all_img_names, report_id = extract_data()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d67d5ca8-f99a-4baf-8748-4aa99efbd09b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>